In [1]:
import threading
import sys

if sys.version_info >= (3, 0):
    import queue as Queue
else:
    import Queue

import threading
import sys

if sys.version_info >= (3, 0):
    import queue as Queue
else:
    import Queue
'''
1. 使用线程进行流水线操作，参见tensorlfow的input pipline的流水线设计
2. 使用队列，生产者消费者模型
'''
class BackgroundGenerator(threading.Thread):
    def __init__(self, generator, max_prefetch=1):
        """
        This function transforms generator into a background-thead generator.
        :param generator: generator or genexp or any
        It can be used with any minibatch generator.
        It is quite lightweight, but not entirely weightless.
        Using global variables inside generator is not recommended (may rise GIL and zero-out the benefit of having a background thread.)
        The ideal use case is when everything it requires is store inside it and everything it outputs is passed through queue.
        There's no restriction on doing weird stuff, reading/writing files, retrieving URLs [or whatever] wlilst iterating.
        :param max_prefetch: defines, how many iterations (at most) can background generator keep stored at any moment of time.
        Whenever there's already max_prefetch batches stored in queue, the background process will halt until one of these batches is dequeued.
        !Default max_prefetch=1 is okay unless you deal with some weird file IO in your generator!
        Setting max_prefetch to -1 lets it store as many batches as it can, which will work slightly (if any) faster, but will require storing
        all batches in memory. If you use infinite generator with max_prefetch=-1, it will exceed the RAM size unless dequeued quickly enough.
        """
        threading.Thread.__init__(self)
        self.queue = Queue.Queue(max_prefetch)
        self.generator = generator
        self.daemon = True
        self.start()

    def run(self):
        for item in self.generator:
            self.queue.put(item)
        self.queue.put(None)

    def next(self):
        next_item = self.queue.get()
        if next_item is None:
            raise StopIteration
        return next_item

    # Python 3 compatibility
    def __next__(self):
        return self.next()

    def __iter__(self):
        return self
    
#decorator
class background:
    def __init__(self, max_prefetch=1):
        self.max_prefetch = max_prefetch
    def __call__(self, gen):
        def bg_generator(*args,**kwargs):
            return BackgroundGenerator(gen(*args,**kwargs), max_prefetch=self.max_prefetch)
        return bg_generator

In [2]:
import numpy as np
import time

def iterate_minibatches(n_batches, batch_size=10):
    for b_i in range(n_batches):
        time.sleep(0.1) #here it could read file or SQL-get or do some math
        X = np.random.normal(size=[batch_size,20])
        y = np.random.randint(0,2,size=batch_size)
        yield X,y

In [3]:
%%time

for b_x,b_y in iterate_minibatches(50):
    #training
    time.sleep(0.1) #here it could use GPU for example
    print('=',end="")
    
print()

CPU times: user 77 ms, sys: 21.8 ms, total: 98.7 ms
Wall time: 10.3 s


In [4]:
%%time

for b_x,b_y in BackgroundGenerator(iterate_minibatches(50)):
    #training
    time.sleep(0.1) #here it could use some GPU
    print('=',end="")
print()

CPU times: user 60.4 ms, sys: 20 ms, total: 80.4 ms
Wall time: 5.25 s


In [5]:
@background(max_prefetch=3)
def bg_iterate_minibatches(n_batches, batch_size=10):
    for b_i in range(n_batches):
        time.sleep(0.1) #here it could read file or SQL-get or do some math
        X = np.random.normal(size=[batch_size,20])
        y = np.random.randint(0,2,size=batch_size)
        yield X,y

In [6]:
%%time

for b_x,b_y in bg_iterate_minibatches(50):
    #training
    time.sleep(0.1)#you guessed it
    print('=',end="")
print()

CPU times: user 61.6 ms, sys: 20 ms, total: 81.6 ms
Wall time: 5.25 s


# CPU时间

进程时间也称CPU时间，用以度量进程使用的中央处理器资源。进程时间以时钟嘀嗒计算

实际时间指实际流逝的时间；用户时间和系统时间指特定进程使用的CPU时间

* real time是从进行开始执行到完成所经历的墙上时钟时间（wall clock）时间，包括其他进程使用的时间片（time slice）和本进程耗费在阻塞（如等待I/O操作完成）上的时间

* user time是进程执行用户态代码（内核外）耗费的CPU时间，仅统计该进程执行时实际使用的CPU时间，而不计入其他进程使用的时间片和本进程阻塞的时间

* sys time 是该进程在内核态运行所耗费的CPU时间，即内核执行系统调用所使用的CPU时间

CPU总时间（user + sys）是CPU执行用户进程操作和内核（代表用户进程执行）系统调用所耗时间的总和，即该进程（包括线程和子进程）所使用的实际CPU时间。若程序循环遍历数组，则增加用户CPU时间；若程序执行exec或fork等系统调用，则增加系统CPU时间。
在多核处理器机器上，若进程含有多个线程或通过fork调用创建子进程，则实际时间可能小于CPU总时间，因为不同线程或进程可并行执行，但其时间会计入主进程的CPU总时间。若程序在某段时间处于等待状态而并未执行，则实际时间可能大于CPU总时间：

- real < CPU  表明进程为计算密集型（CPU bound），利用多核处理器的并行执行优势
- real ≈ CPU  表明进程为计算密集型，未并行执行
- real > CPU  表明进程为I/O密集型 （I/O bound），多核并行执行优势并不明显